In [1]:
# Import necessary Python libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 1.Import Data

In [3]:
# Load and clean the dataset
df = pd.read_excel('final_housing.xlsx', sheet_name="Data")
# Fill missing values with column means
df_filled = df.fillna(df.mean(numeric_only=True))

# Check for any remaining NaNs
print("Remaining NaNs per column:\n", df_filled.isna().sum())

Remaining NaNs per column:
 City                    0
Date                    0
RentHousePrice_Ratio    0
Mortgage                0
Crime_Rate              0
Unemployment_Rate       0
MortgagePay_PerMonth    0
School_Search_Number    0
Housing_Price_Cut       0
Building_Permits        0
dtype: int64


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   City                  1050 non-null   object        
 1   Date                  1050 non-null   datetime64[ns]
 2   RentHousePrice_Ratio  1050 non-null   float64       
 3   Mortgage              1050 non-null   float64       
 4   Crime_Rate            1050 non-null   float64       
 5   Unemployment_Rate     1042 non-null   float64       
 6   MortgagePay_PerMonth  1050 non-null   float64       
 7   School_Search_Number  1050 non-null   int64         
 8   Housing_Price_Cut     840 non-null    float64       
 9   Building_Permits      1050 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 82.2+ KB


In [5]:
print(df['Housing_Price_Cut'].isna().sum())

210


In [6]:
df_filled['Housing_Price_Cut'] = df_filled['Housing_Price_Cut'].fillna(df_filled['Housing_Price_Cut'].median())

In [7]:
print(df_filled.head())     # First 5 rows
print(df_filled.tail())     # Last 5 rows
print(df_filled.dtypes)     # Data types of columns


              City       Date  RentHousePrice_Ratio  Mortgage  Crime_Rate  \
0  Los Angeles, CA 2016-06-30              0.005120      3.66       737.0   
1  Los Angeles, CA 2016-07-31              0.005136      3.41       737.0   
2  Los Angeles, CA 2016-08-31              0.005184      3.43       737.0   
3  Los Angeles, CA 2016-09-30              0.005261      3.46       737.0   
4  Los Angeles, CA 2016-10-31              0.005318      3.42       737.0   

   Unemployment_Rate  MortgagePay_PerMonth  School_Search_Number  \
0                5.5           2691.238403                    81   
1                5.8           2663.114359                    75   
2                5.7           2645.850984                    81   
3                5.6           2621.161649                    79   
4                5.3           2601.993478                    76   

   Housing_Price_Cut  Building_Permits  
0            0.17463       2741.433193  
1            0.17463       2371.246447  
2    

## 2.EDA

In [ ]:
# Density plot for Rent / House Price
plt.figure(figsize=(8, 5))
sns.kdeplot(df_filled['RentHousePrice_Ratio'], fill=True, color='skyblue', linewidth=2)
plt.title('Density Plot: Rent / House Price Ratio')
plt.xlabel('Rent / House Price')
plt.ylabel('Density')
plt.tight_layout()
plt.show()


In [ ]:
# Select numeric data
df_numeric = df_filled.select_dtypes(include=[np.number])

# Calculate layout
num_vars = df_numeric.shape[1]
cols = 3
rows = int(np.ceil(num_vars / cols))

# Plot all histograms
df_numeric.hist(bins=20, figsize=(15, 5 * rows), layout=(rows, cols), edgecolor='black')
plt.suptitle("Histograms for All Numeric Variables", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Density plots for all numeric features
# Select only numeric columns for density plot
df_numeric = df_filled.select_dtypes(include=[np.number])

# Automatically calculate layout
n_vars = len(df_numeric.columns)
cols = 3
rows = int(np.ceil(n_vars / cols))

# Plot
df_numeric.plot(kind='density', subplots=True, layout=(rows, cols), figsize=(15, 5 * rows), sharex=False)
plt.suptitle("Density Plots for Numeric Variables")
plt.tight_layout()
plt.show()


In [ ]:
df_filled.set_index('Date')['RentHousePrice_Ratio'].plot()


## 3.Data Preprocessing

In [ ]:
df_filled[:10]

In [ ]:
column_names = df.columns
print(column_names)

In [ ]:
df_corr = df_filled[['Mortgage','Crime_Rate', 'Unemployment_Rate','School_Search_Number','Housing_Price_Cut','Building_Permits','RentHousePrice_Ratio']]

df_corr[:10]

### MultiCollinearity

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# If X_scaled is a numpy array, convert it back to a DataFrame


X_df = pd.DataFrame(df_corr, columns=df_corr.columns)

# Compute correlation matrix
corr_matrix = X_df.corr()

# Plot heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
# everything except target
y = df_filled['RentHousePrice_Ratio']
X = df_filled.drop(columns=['Date','RentHousePrice_Ratio','MortgagePay_PerMonth','City'])

## Interpretation
1.Mortgage vs Unemployment_Rate: There's a moderate negative correlation (-0.32), indicating that areas with higher unemployment tend to have lower mortgage values—possibly due to reduced purchasing power or loan eligibility.

2.Building_Permits vs RentHousePrice_Ratio: There is a moderate positive correlation (0.38), suggesting that as more building permits are issued (indicating construction activity), the rent-to-price ratio tends to increase—possibly because new developments may target rental markets first.

3.School_Search_Number vs Crime_Rate: A moderate positive correlation (0.38) exists, which may indicate that in areas with higher crime rates, people are more actively searching for schools—perhaps looking for safer or better school districts.

4.Housing_Price_Cut vs Mortgage: There's a moderate positive correlation (0.36), which could imply that areas with more mortgage activity also see more frequent price cuts, potentially reflecting buyer negotiation or market corrections.

5.RentHousePrice_Ratio shows weak correlations: It has very low correlations with most variables (all around ±0.01 to 0.38), suggesting that the rent-to-house-price ratio is influenced by multiple smaller factors or other external variables not shown in this dataset.

## 5.Linear Regression

In [ ]:
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1. Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Scale only X
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Train the model (no scaling y)
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# 4. Predict (on scaled X_test)
y_pred = lr_model.predict(X_test_scaled)

# 5. Evaluate using original y_test
lr_score = lr_model.score(X_test_scaled, y_test)

# Calculate errors
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# 6. Print results
print(f"R² score on test set: {lr_score:.4f}")
print(f"MAE: {mae:.10f}")
print(f"MSE: {mse:.10f}")


In [ ]:
lr_coefficients = pd.Series(lr_model.coef_, index=X.columns)
print(f"R² score on test set: {lr_score:.4f}") # Closer to 1 is better
print("Linear Regression Coefficients:") # A positive coefficient means that as the independent variable increases, the ratio tends to increase
print(lr_coefficients) # A negative coefficient means that the variable has an inverse relationship

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

In [ ]:
# 1. Get intercept
intercept = lr_model.intercept_

# 2. Get coefficients with feature names
coefficients = pd.Series(lr_model.coef_, index=X.columns)

# 3. Print the regression equation
print(f"Linear Regression Equation:\n")

equation = f"y = {intercept:.6f}"

for feature, coef in coefficients.items():
    if coef >= 0:
        equation += f" + {coef:.6f}*{feature}"
    else:
        equation += f" - {abs(coef):.6f}*{feature}"

print(equation)


In [ ]:
# 2. Display the correlation table
print("Correlation Matrix:\n")
print(corr_matrix)

# 3. Optionally round and sort by correlation with the target variable
target = 'RentHousePrice_Ratio'
correlation_with_target = corr_matrix[target].sort_values(ascending=False)
print("\nCorrelation of features with RentHousePrice_Ratio:")
print(correlation_with_target)


## Interpretation:
Low R² Score (0.1781):
Your model explains only 17.8% of the variance in the target variable (RentHousePrice_Ratio). This means most of the variation is still unaccounted for—indicating a weak model fit.

Error Metrics:

MAE (0.00053): On average, your model's predictions are off by about 0.00053, which may seem small, but relative to the prediction scale (~0.004), it’s significant.

MSE (0.00000043): Squared errors are low, but this is expected with a small target value range.

Actual vs Predicted Scatter Plot:

There is a general upward trend, but the points are widely scattered, showing inconsistent prediction accuracy.

Your model is struggling to align predictions closely with actual values, especially at higher target values.

## OLS Regression

In [ ]:
# OLS regression
import statsmodels.api as sm

# Assume X and y are already defined as:
# y = df['RentHousePrice_Ratio']
# X = df[['Schools', 'Jobs', 'Business']] or any variables you choose

# Add a constant (intercept term)
X_sm = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X_sm).fit()

# Print the summary table (like in your image)
print(model.summary())


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr_model, X_train_scaled, y_train, cv=5, scoring='r2')
print("Cross-validation R² scores:", scores)
print("Average CV R²:", scores.mean())

## Interpretations:

1.The model explains only 16.7% of the variation in the rent-to-house-price ratio, indicating a weak linear relationship.

2.All variables are statistically significant, with Building_Permits having the strongest positive effect and Mortgage and Crime_Rate having negative effects.

3.The high condition number (28,900) suggests possible multicollinearity among the predictors, which could affect the stability of the model.



## 6.K-means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 1. Scale the features (if not already scaled)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Use full X, not just X_train

# 2. Find the best k (number of clusters) using Elbow Method and Silhouette Score
inertia = []
silhouette_scores = []
k_values = range(2, 11)  # Test from 2 to 10 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels))

# 3. Plot Inertia (Elbow curve)
plt.figure(figsize=(8, 4))
plt.plot(k_values, inertia, marker='o')
plt.title('Elbow Method (Inertia vs k)')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

# 4. Plot Silhouette Scores
plt.figure(figsize=(8, 4))
plt.plot(k_values, silhouette_scores, marker='o', color='orange')
plt.title('Silhouette Score vs Number of clusters (k)')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()


## HyperParameter Tuning - Elbow Method

In [ ]:
# 5. Choose the best k manually based on plots (for example, k=3)
optimal_k = 3
# You can adjust this based on plots

# 6. Final KMeans with the selected k
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
final_labels = final_kmeans.fit_predict(X_scaled)

# 7. Final silhouette score
final_silhouette = silhouette_score(X_scaled, final_labels)

print(f"Final KMeans Model with k={optimal_k}")
print(f"Final Silhouette Score: {final_silhouette:.4f}")


## Interpretations

lbow Plot Insight
The elbow in the Inertia vs k graph appears around k=3 to 4, suggesting that 3 or 4 clusters is a good balance between compactness and simplicity. Beyond that point, the decrease in inertia slows down, meaning adding more clusters gives diminishing returns.

Silhouette Score Insight
The highest silhouette score (~0.256) occurs at k=6 and k=10, indicating that those clusterings have the best cohesion and separation. However, the silhouette score at k=3 (0.2289) is still acceptable, though not optimal.

Final Model Choice (k=3)
Using k=3 for the final KMeans model gives a moderate silhouette score, meaning the clusters are reasonably well-formed but not very distinct. This choice may prioritize interpretability or simplicity over accuracy.


## 7.Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# 1. Split dataset (use unscaled X and y first)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Scale features X
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Scale target y
y_train_scaled = y_train * 1000
y_test_scaled = y_test * 1000

# 4. Train Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train_scaled)

# 5. Predict
y_pred_scaled = rf.predict(X_test_scaled)

# 6. Rescale predictions back
y_pred = y_pred_scaled / 1000  # <-- Important to bring it back to real-world scale

# 7. Evaluate model
r2 = r2_score(y_test, y_pred)  # use original y_test (not scaled)
rmse = mean_squared_error(y_test, y_pred) ** 0.5

# 8. Print results
print(f"Random Forest R² Score: {r2:.4f}")
print(f"Random Forest RMSE: {rmse:.6f}")


In [ ]:
# Get feature importances
importances = rf.feature_importances_
features = X.columns

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=importances, y=features)
plt.title("Random Forest Feature Importances")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


### HyperParameter Tuning - Choosing the best estimators

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Define corrected parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']  # Removed 'auto'
}

# Initialize Random Forest
rf = RandomForestRegressor(random_state=42)

# GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=5, scoring='r2', n_jobs=-1, verbose=1)

# Fit on training data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Use best model to predict
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Evaluate
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Manual RMSE

print(f"Tuned Random Forest R² Score: {r2:.4f}")
print(f"Tuned Random Forest RMSE: {rmse:.6f}")


### Interpretation

Model Performance
The Tuned Random Forest model performs extremely well with an R² score of 0.9304, meaning it explains over 93% of the variance in the rent-to-house-price ratio. The low RMSE (0.000192) indicates high prediction accuracy.

Most Important Feature
Crime_Rate is by far the most influential variable in the model, showing the highest feature importance. This suggests that areas with differing crime levels play a critical role in determining rent returns relative to house prices.

Other Influential Features
Schools and Unemployment_Rate also contribute significantly, while Mortgage and Housing_Price_Cut have minimal influence on the model’s predictions.


## 8.PCA Analysis

In [ ]:
# PCA Analysis
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Fit PCA on standardized data

pca = PCA(n_components=0.95)  # retain 95% variance
X_pca = pca.fit_transform(X_train_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
print("PCA Explained Variance Ratio per Component:")
print(explained_variance)

# Optional: Scree plot
plt.figure(figsize=(8, 5))
sns.barplot(x=list(range(1, len(explained_variance) + 1)), y=explained_variance)
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.title("Scree Plot: PCA Components")
plt.tight_layout()
plt.show()


Top 2 Components Explain Over Half the Variance
The first two principal components together explain approximately 55.3% of the total variance (0.2891 + 0.2642), making them the most informative dimensions.

Diminishing Returns After PC3
The explained variance drops notably after the third component. PC3 adds another ~17.3%, but later components contribute less (under 11% each), suggesting reduced informational value.

Dimensionality Reduction Opportunity
You could reduce the dataset to 3 or 4 principal components while retaining a majority (over 83%) of the data’s variability, which can improve efficiency without major information loss.